 !pip install requests langchain
 !pip install streamlit langchain llama-cpp-python requests
 !pip install llama-cpp-python
 !pip install requests langchain langchain-community
 !pip install streamlit
 !pip install crewai_tools exa_py

In [ ]:
import requests
import os
import pandas as pd
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import streamlit as st
from llama_cpp import Llama
import matplotlib.pyplot as plt
import io
from crypto_core import fetch_crypto_price, fetch_crypto_news, generate_summary, plot_price_trend

# Load environment variables

In [3]:

env_path = '/Users/gita/Documents/Code/VS_Code/LLM/.env'  # ✅ Update this path
load_dotenv(dotenv_path=env_path)

ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')

stock_symbol = 'AAPL'  # Example: Apple Inc.

def get_stock_price(symbol):
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'GLOBAL_QUOTE',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY
    }
    response = requests.get(url, params=params)
    data = response.json()
    price = data['Global Quote']['05. price']
    return price

stock_price = get_stock_price(stock_symbol)
print(f"Current price of {stock_symbol}: ${stock_price}")


Current price of AAPL: $201.0000


In [4]:
def get_crypto_price(crypto_id):
    url = f'https://api.coingecko.com/api/v3/simple/price'
    params = {
        'ids': crypto_id,
        'vs_currencies': 'usd'
    }
    response = requests.get(url, params=params)
    data = response.json()
    price = data[crypto_id]['usd']
    return price

crypto_id = 'bitcoin'  # Example: Bitcoin
crypto_price = get_crypto_price(crypto_id)
print(f"Current price of {crypto_id.capitalize()}: ${crypto_price}")


Current price of Bitcoin: $102806


In [ ]:
# # Step 1: Create a directory to store the model
# !mkdir -p /content/models/CapybaraHermes

# # Step 2: Use wget to download the model directly into the folder
# !wget -O /content/models/CapybaraHermes/capybarahermes-2.5-mistral-7b.Q5_K_S.gguf \
# https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q5_K_S.gguf

# Step 3: Verify that the model was downloaded successfully
model_path = "/Users/gita/Documents/Code/VS_Code/LLM/capybarahermes-2.5-mistral-7b.Q5_K_S.gguf"

if os.path.exists(model_path):
    print(f"✅ Model downloaded successfully at: {model_path}")
else:
    print("❌ Download failed. Please check the URL or permissions.")


✅ Model downloaded successfully at: /Users/gita/Documents/Code/VS_Code/LLM/capybarahermes-2.5-mistral-7b.Q5_K_S.gguf


In [ ]:
llm = Llama(
    model_path=model_path,
    n_ctx=2048,       # Adjust as needed
    n_threads=4,      # You can increase this based on Colab runtime
    verbose=True
)

llama_model_load_from_file_impl: using device Metal (Apple M4) - 10922 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/gita/Documents/Code/VS_Code/LLM/capybarahermes-2.5-mistral-7b.Q5_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = argilla_capybarahermes-2.5-mistral-7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_load

In [8]:
# Define the prompt separately to allow easy modification without re-running the model
question_prompt = "Q: What is a transformer model?\nA:"

In [9]:
response = llm(question_prompt, max_tokens=100)
print(response["choices"][0]["text"])

llama_perf_context_print:        load time =    5414.65 ms
llama_perf_context_print: prompt eval time =    5414.18 ms /    13 tokens (  416.48 ms per token,     2.40 tokens per second)
llama_perf_context_print:        eval time =    7936.47 ms /    99 runs   (   80.17 ms per token,    12.47 tokens per second)
llama_perf_context_print:       total time =   13364.47 ms /   112 tokens


 A transformer model is a type of artificial neural network architecture that is used for tasks such as natural language processing, machine translation, and text generation. It consists of an encoder and a decoder, which are both made up of a series of stacked self-attention mechanisms and feed-forward neural networks. The encoder takes in an input sequence and produces a fixed-length representation, while the decoder generates an output sequence based on the representation produced by the encoder. The transform


In [11]:
def fetch_crypto_price(crypto_id="bitcoin"):
    url = 'https://api.coingecko.com/api/v3/simple/price'
    params = {'ids': crypto_id, 'vs_currencies': 'usd'}
    res = requests.get(url, params=params).json()
    return res.get(crypto_id, {}).get("usd", None)

def fetch_crypto_news():
    url = "https://min-api.cryptocompare.com/data/v2/news/?lang=EN"
    res = requests.get(url).json()
    return [article['title'] for article in res['Data'][:5]]

#================================
#📊 DATA VISUALIZATION
#================================

def plot_price_trend(df, crypto_id):
    plt.figure(figsize=(10, 4))
    plt.plot(df['date'], df['price'], marker='o')
    plt.title(f"{crypto_id.capitalize()} - Price Trend (7 days)")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.grid(True)

    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    return buf

# def plot_price_trend(price_df):
#     plt.figure(figsize=(10,5))
#     plt.plot(price_df['date'], price_df['price'], marker='o')
#     plt.title(f"Price Trend for {crypto_id.title()}")
#     plt.xlabel("Date")
#     plt.ylabel("Price in USD")
#     plt.xticks(rotation=45)
#     plt.grid(True)
#     plt.tight_layout()
#     plt.savefig("price_trend.png")
#     plt.close()
# # ================================
# Streamlit App
# ================================
# st.title("Crypto Price Tracker")
# st.sidebar.header("Settings")
# crypto_id = st.sidebar.text_input("Enter Crypto ID (e.g., bitcoin, ethereum):", "bitcoin")
# if st.sidebar.button("Fetch Price"):
#     price = fetch_crypto_price(crypto_id)
#     if price:
#         st.success(f"Current price of {crypto_id.title()}: ${price}")
#     else:
#         st.error("Failed to fetch price. Please check the Crypto ID.")
# if st.sidebar.button("Fetch News"):
#     news = fetch_crypto_news()
#     if news:
#         st.subheader("Latest Crypto News")
#         for article in news:
#             st.write(f"- {article}")
#     else:
#         st.error("Failed to fetch news.")
# # Example DataFrame for plotting


In [12]:
def generate_summary(news_list):
    combined = "\n".join(news_list)
    prompt = f"""Summarize the following cryptocurrency news headlines in a few insightful bullet points:\n{combined}"""
    result = llm(prompt, max_tokens=300)
    return result

In [ ]:
st.set_page_config(page_title="🧠 AI-Powered Crypto Dashboard", layout="wide")
st.title("📈 Real-Time Crypto Intelligence System")

# --- Sidebar ---
with st.sidebar:
    st.header("📌 Settings")
    selected_coin = st.selectbox("Choose a Cryptocurrency", ["bitcoin", "ethereum", "dogecoin"])
    query = st.text_area("Ask a question to the model", "What’s the latest update on Ethereum?")

# --- Main UI ---
col1, col2 = st.columns([2, 1])

def fetch_historical_price(crypto_id):
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
    params = {"vs_currency": "usd", "days": 7}
    resp = requests.get(url, params=params)
    if resp.status_code == 429:
        return None, "Rate limit exceeded. Please try again later."
    if resp.status_code != 200:
        return None, f"Failed to fetch data. HTTP Status: {resp.status_code}"
    data = resp.json()
    prices = data.get("prices", [])
    df = pd.DataFrame(prices, columns=["timestamp", "price"])
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df[["date", "price"]], None

with col1:
    st.subheader("📊 Price Trend")
    price_df, err = fetch_historical_price(selected_coin)
    if err:
        st.error(err)
    else:
        st.line_chart(price_df.set_index("date")["price"])

with col2:
    st.subheader("📰 Latest News")
    try:
        news = fetch_crypto_news(selected_coin)
        for item in news:
            st.markdown(f"- {item}")
    except Exception as e:
        st.error(f"Failed to fetch news: {e}")

# --- Query Result ---
if query:
    st.subheader("🤖 LLM Response")
    try:
        response = generate_summary("\n".join(news))
        st.success(response)
    except Exception as e:
        st.error(f"Failed to generate summary: {e}")


2025-06-22 10:51:51.575 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 10:51:51.575 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 10:51:51.632 
  command:

    streamlit run /Users/gita/Library/Python/3.13/lib/python/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-22 10:51:51.633 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 10:51:51.633 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 10:51:51.633 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-22 10:51:51.633 Thread 'MainThread': missing ScriptRunContext! This warning can be ign

In [ ]:
# Run the streamlit app (this will block the notebook cell)
# You will typically see a link to access the app in the output
!streamlit run crypto_dashboard.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.20.10.2:8501

llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_met

In [15]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.150.105:8501

2025-06-22 10:52:39.720 Uncaught app execution
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/streamlit/runtime/scriptrunner/exec_code.py", line 121, in exec_func_with_error_handling
    result = func()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 645, in code_to_exec
    exec(code, module.__dict__)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/gita/Documents/Code/VS_Code/LLM/app.py", line 8, in <module>
    import plotly.express as px
ModuleNotFoundError: No module named 'plotly'
^C
